In [ ]:
!pip install spacy_transformers
!pip install -U spacy

In [2]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm   
import json

In [3]:
spacy.__version__

'3.7.2'

In [4]:
cv_data= json.load(open('traindata.json'))

In [5]:
len(cv_data)

29

In [24]:
#!python -m spacy init fill-config base_config.cfg config.cfg
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [7]:
cv_data[0]

{'id': 2,
 'annotations': [{'id': 1,
   'completed_by': 1,
   'result': [{'value': {'start': 10,
      'end': 63,
      'text': '',
      'labels': ['Berufserfahrung']},
     'id': 'BkIyY427mB',
     'from_name': 'label',
     'to_name': 'text',
     'type': 'labels',
     'origin': 'manual'},
    {'value': {'start': 78,
      'end': 87,
      'text': '.Net (C#)',
      'labels': ['Skills']},
     'id': 'TDGtnx1smh',
     'from_name': 'label',
     'to_name': 'text',
     'type': 'labels',
     'origin': 'manual'}],
   'was_cancelled': False,
   'ground_truth': False,
   'created_at': '2023-12-30T22:56:21.301770Z',
   'updated_at': '2023-12-30T22:56:21.301770Z',
   'draft_created_at': '2023-12-30T22:54:57.380759Z',
   'lead_time': 86.25800000000001,
   'prediction': {},
   'result_count': 0,
   'unique_id': '8b9ae70f-6fbe-43e3-a306-9bac8c047b2b',
   'import_id': None,
   'last_action': None,
   'task': 2,
   'project': 1,
   'updated_by': 1,
   'parent_prediction': None,
   'parent_ann

In [30]:
#should we use entities or annotations?

def get_spacy_doc(file, data):
    nlp = spacy.blank('de')
    db = DocBin()
    
    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)
        annot = annot['annotations']

        ents = []
        entity_indices = []

        for start, end, label in annot:
            skip_entity = False
            for idx in range(start, end):
                if idx in entity_indices:
                    skip_entity = True
                    break 
            if skip_entity==True:
                continue
            entity_indices = entity_indices + list(range(start, end))

            try:
                span = doc.char_span(start, end, label=label, alignment_mode='strict')
            except: 
                continue

            if span is None:
                err_data = str([start, end]) + "   " + str(text) + "\n"
                file.write(err_data)

            else:
                ents.append(span)
                
                try:
                    doc.ents = ents
                    db.add(doc)
                except: 
                    pass

                return db 


In [31]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.3)

In [32]:
len(train), len(test)

(20, 9)

In [33]:
file = open('error.txt', 'w')

db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

file.close()

  0%|          | 0/20 [00:00<?, ?it/s]


ValueError: too many values to unpack (expected 2)

In [34]:
len(db.tokens)

NameError: name 'db' is not defined